 <div style="background-color:#004d99; color:white; padding:30px; BORDER-radius:10px; text-align:center;">
  <h1 style="margin:0; font-size:1.8em;">ENTERPRISE ANALYTICS DASHBOARD</h1>
</div>

In [1]:
import subprocess
import time
import os
import signal
import sys

# Streamlit app code
app_code = '''

import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
from datetime import date, timedelta
import calendar

# Set page configuration
st.set_page_config(
    page_title="Enterprise Analytics Dashboard",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Apply custom CSS for professional styling
st.markdown("""
<style>
    /* Main styling */
    .main .block-container {
        padding-top: 2rem;
        padding-bottom: 2rem;
    }
    
    /* Header styling */
    .dashboard-header {
        display: flex;
        justify-content: space-between;
        align-items: center;
        padding: 1rem 0;
        border-bottom: 1px solid #e6e6e6;
        margin-bottom: 2rem;
    }
    
    /* Card styling */
    .metric-card {
        background-color: white;
        border-radius: 10px;
        box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08);
        padding: 1.5rem;
        height: 140px;
        display: flex;
        flex-direction: column;
        justify-content: center;
        transition: transform 0.2s;
    }
    
    .metric-card:hover {
        transform: translateY(-5px);
        box-shadow: 0 6px 16px rgba(0, 0, 0, 0.12);
    }
    
    .metric-value {
        font-size: 2.2rem;
        font-weight: 700;
        color: #2c3e50;
        margin-bottom: 0.2rem;
    }
    
    .metric-label {
        font-size: 0.95rem;
        color: #7f8c8d;
        margin-bottom: 0.5rem;
        font-weight: 500;
    }
    
    .metric-change {
        font-size: 0.85rem;
        display: flex;
        align-items: center;
        font-weight: 500;
    }
    
    .positive-change {
        color: #27ae60;
    }
    
    .negative-change {
        color: #e74c3c;
    }
    
    /* KPI Cards */
    .kpi-card {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        border-radius: 12px;
        padding: 1.5rem;
        color: white;
        height: 140px;
        display: flex;
        flex-direction: column;
        justify-content: center;
    }
    
    .kpi-value {
        font-size: 2.5rem;
        font-weight: 700;
        margin-bottom: 0.5rem;
    }
    
    .kpi-label {
        font-size: 1rem;
        opacity: 0.9;
    }
    
    /* Section headers */
    .section-header {
        font-size: 1.4rem;
        font-weight: 600;
        color: #2c3e50;
        margin-bottom: 1.5rem;
        padding-bottom: 0.5rem;
        border-bottom: 2px solid #f0f2f6;
    }
    
    /* Custom select boxes */
    .stSelectbox div div div {
        border-radius: 8px;
    }
    
    /* Custom date input */
    .stDateInput div div div {
        border-radius: 8px;
    }
    
    /* Custom buttons */
    .stButton button {
        border-radius: 8px;
        background-color: #667eea;
        color: white;
        font-weight: 500;
        border: none;
        transition: all 0.3s;
    }
    
    .stButton button:hover {
        background-color: #5a6fd5;
        color: white;
    }
    
    /* Custom tabs */
    .stTabs [data-baseweb="tab-list"] {
        gap: 24px;
    }
    
    .stTabs [data-baseweb="tab"] {
        height: 50px;
        white-space: pre-wrap;
        background-color: #f8f9fa;
        border-radius: 8px 8px 0px 0px;
        gap: 1px;
        padding: 15px 25px;
        font-weight: 500;
    }
    
    .stTabs [aria-selected="true"] {
        background-color: #667eea;
        color: white;
    }
    
    /* Custom dataframe */
    .dataframe {
        border-radius: 8px;
        overflow: hidden;
        box-shadow: 0 4px 12px rgba(0, 0, 0, 0.05);
    }
    
    /* Custom metric labels in dataframe */
    .dataframe th {
        background-color: #667eea;
        color: white;
    }
    
    /* Custom expander */
    .streamlit-expanderHeader {
        font-weight: 600;
        font-size: 1.1rem;
        color: #2c3e50;
    }
</style>
""", unsafe_allow_html=True)

# Generate sample data
@st.cache_data
def generate_sales_data():
    dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')
    categories = ['Electronics', 'Clothing', 'Home & Kitchen', 'Books', 'Sports']
    regions = ['North America', 'Europe', 'Asia', 'South America', 'Africa']
    
    data = []
    for date in dates:
        for category in categories:
            for region in regions:
                sales = np.random.randint(100, 5000)
                profit = sales * np.random.uniform(0.1, 0.4)
                units = np.random.randint(1, 100)
                data.append({
                    'Date': date,
                    'Category': category,
                    'Region': region,
                    'Sales': sales,
                    'Profit': profit,
                    'Units': units
                })
    
    return pd.DataFrame(data)

@st.cache_data
def generate_customer_data():
    segments = ['New', 'Returning', 'Loyal', 'At Risk']
    channels = ['Organic', 'Email', 'Social', 'Direct', 'Referral']
    
    data = []
    for segment in segments:
        for channel in channels:
            customers = np.random.randint(100, 2000)
            conversion = np.random.uniform(0.01, 0.15)
            clv = np.random.randint(100, 2000)
            data.append({
                'Segment': segment,
                'Channel': channel,
                'Customers': customers,
                'Conversion Rate': conversion,
                'CLV': clv
            })
    
    return pd.DataFrame(data)

@st.cache_data
def generate_financial_data():
    months = pd.date_range(start='2023-01-01', end='2023-12-31', freq='M')
    metrics = ['Revenue', 'Expenses', 'Profit', 'ROI', 'CAC']
    
    data = []
    for month in months:
        for metric in metrics:
            if metric == 'Revenue':
                value = np.random.randint(500000, 1000000)
            elif metric == 'Expenses':
                value = np.random.randint(300000, 600000)
            elif metric == 'Profit':
                value = np.random.randint(100000, 400000)
            elif metric == 'ROI':
                value = np.random.uniform(0.1, 0.4)
            else:  # CAC
                value = np.random.randint(50, 200)
            data.append({
                'Month': month,
                'Metric': metric,
                'Value': value
            })
    
    return pd.DataFrame(data)

# Create the dashboard
def main():
    # Header section
    col1, col2, col3 = st.columns([1, 2, 1])
    with col1:
        st.markdown("<h1 style='color: #2c3e50; margin-bottom: 0;'>Enterprise Analytics</h1>", unsafe_allow_html=True)
        st.markdown("<p style='color: #7f8c8d; margin-top: 0;'>Executive Dashboard</p>", unsafe_allow_html=True)
    
    with col2:
        st.write("")  # Spacer
    
    with col3:
        selected_date = st.date_input(
            "Select Date",
            value=date.today(),
            min_value=date(2023, 1, 1),
            max_value=date.today()
        )
    
    # Filters row
    st.markdown("---")
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        view_option = st.selectbox(
            "View By",
            ["Daily", "Weekly", "Monthly", "Quarterly"]
        )
    
    with col2:
        category_filter = st.multiselect(
            "Category",
            ["Electronics", "Clothing", "Home & Kitchen", "Books", "Sports"],
            default=["Electronics", "Clothing", "Home & Kitchen"]
        )
    
    with col3:
        region_filter = st.selectbox(
            "Region",
            ["All Regions", "North America", "Europe", "Asia", "South America", "Africa"]
        )
    
    with col4:
        metric_option = st.selectbox(
            "Primary Metric",
            ["Sales", "Profit", "Units", "Conversion Rate"]
        )
    
    st.markdown("---")
    
    # KPI Cards
    st.markdown("### Performance Overview")
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        st.markdown("""
        <div class="kpi-card">
            <div class="kpi-value">$4.2M</div>
            <div class="kpi-label">Total Revenue</div>
            <div class="metric-change positive-change">↑ 12.4% from last period</div>
        </div>
        """, unsafe_allow_html=True)
    
    with col2:
        st.markdown("""
        <div class="kpi-card" style="background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%);">
            <div class="kpi-value">$1.8M</div>
            <div class="kpi-label">Total Profit</div>
            <div class="metric-change positive-change">↑ 8.7% from last period</div>
        </div>
        """, unsafe_allow_html=True)
    
    with col3:
        st.markdown("""
        <div class="kpi-card" style="background: linear-gradient(135deg, #ff416c 0%, #ff4b2b 100%);">
            <div class="kpi-value">24.8%</div>
            <div class="kpi-label">Profit Margin</div>
            <div class="metric-change positive-change">↑ 2.1% from last period</div>
        </div>
        """, unsafe_allow_html=True)
    
    with col4:
        st.markdown("""
        <div class="kpi-card" style="background: linear-gradient(135deg, #6a11cb 0%, #2575fc 100%);">
            <div class="kpi-value">42.5K</div>
            <div class="kpi-label">Units Sold</div>
            <div class="metric-change positive-change">↑ 5.3% from last period</div>
        </div>
        """, unsafe_allow_html=True)
    
    # Main content area
    col1, col2 = st.columns([7, 3])
    
    with col1:
        # Sales trend chart
        st.markdown("### Sales Trend")
        
        # Generate sample time series data
        dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='M')
        sales_values = np.random.randint(500000, 1000000, size=len(dates))
        profit_values = sales_values * np.random.uniform(0.2, 0.3, size=len(dates))
        
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        
        fig.add_trace(
            go.Scatter(x=dates, y=sales_values, name="Revenue", line=dict(color="#667eea", width=3)),
            secondary_y=False,
        )
        
        fig.add_trace(
            go.Scatter(x=dates, y=profit_values, name="Profit", line=dict(color="#11998e", width=3)),
            secondary_y=True,
        )
        
        fig.update_layout(
            height=400,
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)',
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
            hovermode="x unified"
        )
        
        fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=1, gridcolor='#f0f2f6')
        fig.update_yaxes(title_text="Revenue ($)", secondary_y=False, showgrid=True, gridwidth=1, gridcolor='#f0f2f6')
        fig.update_yaxes(title_text="Profit ($)", secondary_y=True, showgrid=False)
        
        st.plotly_chart(fig, use_container_width=True)
        
        # Category performance
        st.markdown("### Category Performance")
        
        categories = ['Electronics', 'Clothing', 'Home & Kitchen', 'Books', 'Sports']
        sales_data = [4200000, 3800000, 2900000, 1500000, 2100000]
        profit_margins = [28.5, 22.3, 25.7, 18.9, 30.2]
        
        fig = go.Figure()
        fig.add_trace(go.Bar(
            x=categories,
            y=sales_data,
            name="Sales",
            marker_color=['#667eea', '#764ba2', '#11998e', '#ff416c', '#6a11cb']
        ))
        
        fig.add_trace(go.Scatter(
            x=categories,
            y=profit_margins,
            name="Profit Margin (%)",
            yaxis="y2",
            mode="lines+markers",
            line=dict(color="#ff7e5f", width=3),
            marker=dict(size=8)
        ))
        
        fig.update_layout(
            height=400,
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)',
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
            yaxis=dict(title="Sales ($)"),
            yaxis2=dict(title="Profit Margin (%)", overlaying="y", side="right"),
            hovermode="x unified"
        )
        
        st.plotly_chart(fig, use_container_width=True)
    
    with col2:
        # Regional performance
        st.markdown("### Regional Performance")
        
        regions = ['North America', 'Europe', 'Asia', 'South America', 'Africa']
        sales = [2200000, 1800000, 2100000, 900000, 600000]
        
        fig = px.pie(
            values=sales,
            names=regions,
            color_discrete_sequence=px.colors.sequential.Blues_r
        )
        
        fig.update_traces(
            textposition='inside',
            textinfo='percent+label',
            hovertemplate="<b>%{label}</b><br>Sales: $%{value:,.0f}<br>Percentage: %{percent}"
        )
        
        fig.update_layout(
            height=300,
            showlegend=False,
            margin=dict(l=10, r=10, t=10, b=10)
        )
        
        st.plotly_chart(fig, use_container_width=True)
        
        # Customer metrics
        st.markdown("### Customer Metrics")
        
        metrics = ['Acquisition', 'Retention', 'Conversion', 'Satisfaction']
        values = [82, 75, 64, 88]
        
        fig = go.Figure(go.Bar(
            x=values,
            y=metrics,
            orientation='h',
            marker_color=['#667eea', '#764ba2', '#11998e', '#ff416c']
        ))
        
        fig.update_layout(
            height=300,
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)',
            xaxis=dict(range=[0, 100], title="Score"),
            yaxis=dict(autorange="reversed")
        )
        
        st.plotly_chart(fig, use_container_width=True)
        
        # Quick metrics
        st.markdown("### Quick Metrics")
        
        st.markdown("""
        <div class="metric-card">
            <div class="metric-label">Avg. Order Value</div>
            <div class="metric-value">$142.50</div>
            <div class="metric-change positive-change">↑ 3.2% from last month</div>
        </div>
        """, unsafe_allow_html=True)
        
        st.markdown("""
        <div class="metric-card">
            <div class="metric-label">Customer Acquisition Cost</div>
            <div class="metric-value">$87.30</div>
            <div class="metric-change negative-change">↓ 1.5% from last month</div>
        </div>
        """, unsafe_allow_html=True)
    
    # Bottom section with tabs
    st.markdown("---")
    
    tab1, tab2, tab3 = st.tabs(["📈 Detailed Analysis", "🧮 Data Tables", "🔍 Insights"])
    
    with tab1:
        col1, col2 = st.columns(2)
        
        with col1:
            st.markdown("#### Sales by Category Over Time")
            
            # Generate sample data
            months = pd.date_range(start='2023-01-01', end='2023-12-31', freq='M')
            categories = ['Electronics', 'Clothing', 'Home & Kitchen', 'Books', 'Sports']
            
            data = []
            for month in months:
                for category in categories:
                    sales = np.random.randint(100000, 1000000)
                    data.append({
                        'Month': month,
                        'Category': category,
                        'Sales': sales
                    })
            
            df = pd.DataFrame(data)
            
            fig = px.line(
                df,
                x='Month',
                y='Sales',
                color='Category',
                color_discrete_sequence=px.colors.qualitative.Bold
            )
            
            fig.update_layout(
                height=400,
                plot_bgcolor='rgba(0,0,0,0)',
                paper_bgcolor='rgba(0,0,0,0)',
                legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
            )
            
            st.plotly_chart(fig, use_container_width=True)
        
        with col2:
            st.markdown("#### Profitability Analysis")
            
            # Generate sample data
            categories = ['Electronics', 'Clothing', 'Home & Kitchen', 'Books', 'Sports']
            revenue = [4200000, 3800000, 2900000, 1500000, 2100000]
            costs = [3000000, 2950000, 2150000, 1210000, 1460000]
            profit = [r - c for r, c in zip(revenue, costs)]
            
            fig = go.Figure()
            fig.add_trace(go.Bar(
                x=categories,
                y=revenue,
                name='Revenue',
                marker_color='#667eea'
            ))
            fig.add_trace(go.Bar(
                x=categories,
                y=costs,
                name='Costs',
                marker_color='#e74c3c'
            ))
            fig.add_trace(go.Scatter(
                x=categories,
                y=profit,
                name='Profit',
                mode='lines+markers',
                line=dict(color='#27ae60', width=3),
                marker=dict(size=8)
            ))
            
            fig.update_layout(
                barmode='group',
                height=400,
                plot_bgcolor='rgba(0,0,0,0)',
                paper_bgcolor='rgba(0,0,0,0)',
                legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
            )
            
            st.plotly_chart(fig, use_container_width=True)
    
    with tab2:
        st.markdown("#### Detailed Sales Data")
        
        # Generate sample data
        dates = pd.date_range(start='2023-10-01', end='2023-10-15', freq='D')
        categories = ['Electronics', 'Clothing', 'Home & Kitchen']
        regions = ['North America', 'Europe', 'Asia']
        
        data = []
        for d in dates:
            for cat in categories:
                for reg in regions:
                    sales = np.random.randint(10000, 100000)
                    profit = sales * np.random.uniform(0.15, 0.35)
                    data.append({
                        'Date': d,
                        'Category': cat,
                        'Region': reg,
                        'Sales': sales,
                        'Profit': profit,
                        'Margin': profit / sales * 100
                    })
        
        df = pd.DataFrame(data)
        
        # Pivot the data
        pivot_df = df.pivot_table(
            values='Sales',
            index='Date',
            columns=['Region', 'Category'],
            aggfunc='sum'
        ).fillna(0)
        
        # Format the column names
        pivot_df.columns = [f"{col[1]} ({col[0]})" for col in pivot_df.columns]
        
        # Display the dataframe with formatting
        st.dataframe(
            pivot_df.style.format("${:,.0f}").background_gradient(cmap='Blues'),
            use_container_width=True
        )
        
        # Download button
        csv = pivot_df.to_csv().encode('utf-8')
        st.download_button(
            label="Download data as CSV",
            data=csv,
            file_name="sales_data.csv",
            mime="text/csv",
        )
    
    with tab3:
        col1, col2 = st.columns(2)
        
        with col1:
            st.markdown("#### Key Insights")
            
            insights = [
                {"title": "Electronics sales up 15% YoY", "description": "Strong growth in Q3 driven by new product launches", "impact": "High"},
                {"title": "European market showing signs of saturation", "description": "Growth rate declined to 2% from 7% last quarter", "impact": "Medium"},
                {"title": "Customer acquisition costs decreasing", "description": "CAC down 12% due to improved marketing efficiency", "impact": "High"},
                {"title": "Home & Kitchen category outperforming", "description": "28% growth despite overall market slowdown", "impact": "Medium"},
                {"title": "Asia-Pacific region now our #2 market", "description": "Surpassed Europe in Q3 with 32% growth", "impact": "High"}
            ]
            
            for insight in insights:
                impact_color = "#e74c3c" if insight["impact"] == "High" else "#f39c12" if insight["impact"] == "Medium" else "#27ae60"
                
                st.markdown(f"""
                <div style="background-color: white; border-left: 4px solid {impact_color}; padding: 12px; border-radius: 4px; margin-bottom: 12px; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
                    <h4 style="margin: 0 0 8px 0; color: #2c3e50;">{insight['title']}</h4>
                    <p style="margin: 0 0 8px 0; color: #7f8c8d; font-size: 0.9rem;">{insight['description']}</p>
                    <div style="display: inline-block; background-color: {impact_color}20; color: {impact_color}; padding: 2px 8px; border-radius: 4px; font-size: 0.8rem; font-weight: 500;">
                        Impact: {insight['impact']}
                    </div>
                </div>
                """, unsafe_allow_html=True)
        
        with col2:
            st.markdown("#### Forecast vs Actual")
            
            # Generate sample data
            months = pd.date_range(start='2023-01-01', end='2023-12-31', freq='M')
            actual_sales = np.random.randint(800000, 1200000, size=len(months))
            forecast_sales = actual_sales * np.random.uniform(0.85, 1.15, size=len(months))
            
            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=months,
                y=actual_sales,
                name="Actual",
                line=dict(color="#667eea", width=3)
            ))
            fig.add_trace(go.Scatter(
                x=months,
                y=forecast_sales,
                name="Forecast",
                line=dict(color="#95a5a6", width=3, dash='dash')
            ))
            
            fig.update_layout(
                height=500,
                plot_bgcolor='rgba(0,0,0,0)',
                paper_bgcolor='rgba(0,0,0,0)',
                legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
                yaxis=dict(title="Sales ($)")
            )
            
            st.plotly_chart(fig, use_container_width=True)

if __name__ == "__main__":
    main()
'''



# Save app
app_file = "Test_project_12.py"
with open(app_file, "w", encoding="utf-8") as f:
    f.write(app_code)

print("✅ Streamlit app saved as Test_project_12.py")

# Launch Streamlit
try:
    print("🚀 Starting Streamlit server...")
    process = subprocess.Popen([sys.executable, "-m", "streamlit", "run", app_file])
    
    # Wait for startup (adjust as needed)
    time.sleep(5)
    
    print("💡 App is running. Press Ctrl+C to stop.")
    # Keep the script alive until interrupted
    while True:
        time.sleep(1)

except KeyboardInterrupt:
    print("\n🛑 Shutting down Streamlit server...")
    if process.poll() is None:  # If still running
        process.terminate()
        try:
            process.wait(timeout=10)
        except subprocess.TimeoutExpired:
            process.kill()
            process.wait()
    
    #` Optional: remove the generated file
    os.remove(app_file)
    print("🧹 Cleaned up app file and server.")

✅ Streamlit app saved as Test_project_12.py
🚀 Starting Streamlit server...
💡 App is running. Press Ctrl+C to stop.

🛑 Shutting down Streamlit server...
🧹 Cleaned up app file and server.
